In [ ]:
import numpy as np
import networkx as nx

from Simplex2Vec.Simplex2Vec import Simplex2Vec
from Simplex2Vec.utils import check_prediction

# From NetworkX Graph

In [ ]:
# Create a graph
G = nx.fast_gnp_random_graph(n=100, p=0.1, seed=20)

In [ ]:
# Generate Hasse diagram and random walks
order = 2
s2v = Simplex2Vec(G, n_walks=15, walk_length=10, hasse_max_order=order, workers=2)  # keep p=q=1 to have the usual RW

# Embed nodes
model = s2v.fit()  # Any keywords acceptable by gensim.Word2Vec can be passed, `workers` is automatically passed from the Node2Vec constructor

In [ ]:
# Look for most similar nodes
model.wv.most_similar('2')  # Output node names are always strings

In [ ]:
# Can also be made a one-liner
model = Simplex2Vec(G, n_walks=15, walk_length=10,  workers=2).fit()

In [ ]:
# Dump created hasse diagram to a pickle file
s2v.dump_hasse_diagram('hasse_diag.pkl')

### MODEL CAN BE SAVED WITH model.save(), NATIVELY FROM WORD2VEC

In [ ]:
# Create new Simplex2Vec from dumped hasse diagram
s2v = Simplex2Vec.read_hasse_diagram('hasse_diag.pkl', n_walks=15)
model = s2v.fit()

model.wv.most_similar('2')

In [ ]:
# Create external handle for hasse diagram
H = s2v.get_hasse_diagram(copy=False)

print(H is s2v.hasse)

In [ ]:
# Create external deepcopy of hasse diagram
H = s2v.get_hasse_diagram()

print(H is s2v.hasse)

# From External Hasse Diagram

In [ ]:
#Build hasse diagram externally

from Simplex2Vec.utils import read_simplex_json
from Simplex2Vec.simplex2hasse import simplex2hasse_HOexponential

in_file = "./data/sociopatterns_facets/aggr_5min_cliques_InVS15.json"

data = read_simplex_json(in_file)

In [ ]:
g_hasse = simplex2hasse_HOexponential(data, max_order=None)

In [ ]:
# build model from external hasse diagram
s2v = Simplex2Vec.from_hasse_diagram(g_hasse, n_walks=1, walk_length=10)
model = s2v.fit()

In [ ]:
# limit order of imported hasse diagram
s2v = Simplex2Vec.from_hasse_diagram(g_hasse, n_walks=1, walk_length=10, hasse_max_order=2)

In [ ]:
# warning if you try to set a limit higher than what is present in the external hasse diagram
g_hasse = simplex2hasse_HOexponential(data, max_order=2)
print()

s2v = Simplex2Vec.from_hasse_diagram(g_hasse, n_walks=1, walk_length=10, hasse_max_order=3)

## check higher order link prediction

In [ ]:
import numpy as np
import networkx as nx

from Simplex2Vec.Simplex2Vec import Simplex2Vec
from Simplex2Vec.utils import check_prediction

In [ ]:
#Build hasse diagram externally

from Simplex2Vec.utils import read_simplex_json
from Simplex2Vec.simplex2hasse import simplex2hasse_HOexponential

from itertools import compress

in_file = "./data/sociopatterns_facets/aggr_5min_cliques_InVS15.json"
'./'

data = read_simplex_json(in_file)

In [ ]:
frac = 0.8
g_hasse = simplex2hasse_HOexponential(data, max_order=None)
g_hasse_head = simplex2hasse_HOexponential(data[:int(np.round(len(data)*frac))], max_order=None)


In [ ]:
hh = list(g_hasse.nodes)
hhszs = np.array([len(h) for h in hh])
fullSetNodes = list(compress(hh,hhszs==1))
print('Full set has ' + str(len(fullSetNodes)) + ' 0-d nodes')
fullSetTriangles = list(compress(hh,hhszs==3))
print('Full set has ' + str(len(fullSetTriangles)) + ' 2-d triangles')

In [ ]:
hh = list(g_hasse_head.nodes)
hhszs = np.array([len(h) for h in hh])
partialSetNodes = list(compress(hh,hhszs==1))
print('Partial set has ' + str(len(partialSetNodes)) + ' 0-d nodes')
partialSetTriangles = list(compress(hh,hhszs==3))
print('Partial set has ' + str(len(partialSetTriangles)) + ' 2-d triangles')

In [ ]:
testSet = []
for f in fullSetTriangles:
    if not any(list(f <= ex for ex in partialSetTriangles)):
        testSet.append(f)
print(len(testSet))

In [ ]:
s2v = Simplex2Vec.from_hasse_diagram(g_hasse_head, n_walks=1, walk_length=10, hasse_max_order=None)

model = s2v.fit()

In [ ]:
check_prediction(s2v,testSet,metric='euclidean')
check_prediction(s2v,testSet,metric='cosine')

#### Model significantly improves after further training of embedding

In [ ]:
s2v = Simplex2Vec.from_hasse_diagram(g_hasse_head, n_walks=10, walk_length=100, hasse_max_order=None)

model = s2v.fit()

In [ ]:
check_prediction(s2v,testSet,metric='cosine')

check_prediction(s2v,testSet,metric='euclidean')

#### Note: The original hasse_diag graph is not modified by limiting the simplex order in the creation of the Simplex2vec object and can thus be reused.

# Example: Clustering 

The data we use here is the cluster encounters at school. Simplices are formed if students spend time together for sufficient time. The data has labels also for teachers, however we do not know which class they belong to. Thus we specifically exclude them from the data and keep track of their labels in **teachers_labels**.

In [ ]:
import numpy as np
import networkx as nx
import json
from collections import defaultdict

from Simplex2Vec.Simplex2Vec import Simplex2Vec

from Simplex2Vec.utils import read_simplex_json
from Simplex2Vec.simplex2hasse import *

def Data2Simplex(path):
    data = open(path)
    list_simplices = json.load(data)
    list_simplices_fs=[frozenset([str(i) for i in li]) for li in list_simplices]
    return list_simplices_fs

data_dir = "./data/sociopatterns_facets/"
data_file = "aggr_10min_cliques_Thiers13.json"
in_file = os.path.join(data_dir, data_file)

meta_data_path = "./data/sociopatterns_meta/META_cliques_Thiers13.tsv"

data = Data2Simplex(in_file)

# META information
with open(meta_data_path, 'r') as fh:
    f_meta = np.loadtxt(fh, dtype=str, delimiter='\t')
    
teachers_labels = set([])
for x,c in f_meta[:,[0,1]]:
    if c == 'Teachers':
        teachers_labels.add(x)

num_students = 0
classes_list = set([])
for person in f_meta:
    if person[1] != 'Teachers':
        classes_list.add(person[1])
        num_students += 1
        
num_classes = len(classes_list)
print("Number of students", num_students)
print("Number of classes", num_classes)

In [ ]:
# requirement: sklearn and clusim packages . can be installed through pip.
import clusim.sim as sim
from clusim.clustering import Clustering
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.manifold import TSNE

# sample function that clusters the embedding using KMeans clustering. You may use any other clustering algorithm
def clusterer(model, num_classes, teachers_labels = set([])):
    X = []
    X_labels = []

    for u in model.wv.vocab.keys():
        if len(u.split('-')) == 1 and u not in teachers_labels:
            X.append(model[u])
            X_labels.append(u)

    clusterer = KMeans(n_clusters=num_classes)
    pred_cluster_labels = clusterer.fit_predict(X)

    X_labels_pred_clusters = []
    predicted_clusters_dict = defaultdict(list)
    for x,c in zip(X_labels, pred_cluster_labels):
        predicted_clusters_dict[x].append(c)
        X_labels_pred_clusters.append((x,c))

    true_cluster_indices_dict = dict([(c,i) for i,c in enumerate(np.unique(f_meta[:,1]))])

    true_clusters_dict = defaultdict(list)
    for x, c in f_meta[:,[0,1]]:
        if x in predicted_clusters_dict:
            true_clusters_dict[x].append(true_cluster_indices_dict[c])
    
    X_labels_true_clusters = []
    for lbl in X_labels:
        X_labels_true_clusters.append((lbl, true_clusters_dict[lbl][0]))
        
    return X, X_labels_pred_clusters, X_labels_true_clusters

# Output NMI - normalized mutual information of two clusterings
def get_NMI(X_labels_pred_clusters, X_labels_true_clusters):
    warnings.filterwarnings("ignore")
    
    true_clusters_dict = {node: [lbl] for node, lbl in X_labels_true_clusters}
    predicted_clusters_dict = {node: [lbl] for node, lbl in X_labels_pred_clusters}
    
    c_true = Clustering(true_clusters_dict)
    c_pred = Clustering(predicted_clusters_dict)
    NMI = sim.nmi(c_pred, c_true)
    
    return NMI

# find relabelling of the predicted clusters, that match the true labels with max accuracy
def get_max_accuracy_labels(X_labels_pred_clusters, X_labels_true_clusters):
    from tqdm import tqdm
    def accuracy_score(y_pred, y_true):
        score = 0
        for i in range(len(y_pred)):
            if y_pred[i] == y_true[i]:
                score += 1
        score /= float(len(y_pred))
        return score
    
    sorted_predicted_clusters = sorted(X_labels_pred_clusters, key = lambda x: x[0])
    predicted_clusters = [v[1] for v in sorted_predicted_clusters]
    
    sorted_true_clusters = sorted(X_labels_true_clusters, key = lambda x: x[0])
    true_clusters = [v[1] for v in sorted_true_clusters]
    sorted_labels = [v[0] for v in sorted_true_clusters]
    
    max_accuracy = 0
    max_accuracy_clusters = []
    label_permutations_list = list(itertools.permutations(range(max(true_clusters)+1)))
    for perm_map in tqdm(label_permutations_list, desc = "Max accuracy"):
        predicted_clusters_temp = []
        for i in predicted_clusters:
            predicted_clusters_temp.append(perm_map[i])
        if max_accuracy < accuracy_score(predicted_clusters_temp, true_clusters):
            max_accuracy = accuracy_score(predicted_clusters_temp, true_clusters)
            max_accuracy_clusters.clear()
            max_accuracy_clusters = predicted_clusters_temp
            
    max_acc_dict = {x: max_accuracy_clusters[i] for i, x in enumerate(sorted_labels)}
    return max_acc_dict

# plot graphically the embedding
def plot_clusterings(X, X_labels_pred_clusters, X_labels_true_clusters, teachers_labels = set([]),
                     max_accuracy_dict = {}):
    import matplotlib.pyplot as plt
    from matplotlib import cm
    
    X_transform = TSNE(n_components=2).fit_transform(X)
    X_labels = [node for node, lbl in X_labels_pred_clusters]
    if len(max_accuracy_dict):
        cluster_labels_pred = [max_accuracy_dict[node] for node, lbl in X_labels_pred_clusters]
    else:
        cluster_labels_pred = [lbl for node, lbl in X_labels_pred_clusters]
    
    fig, axs = plt.subplots(1,2,figsize = (10,5), dpi = 96)

    # Plot showing the predicted clusters formed
    colors_list = cm.nipy_spectral(np.asarray(cluster_labels_pred).astype(float) / num_classes)
    axs[0].scatter(*zip(*X_transform), marker='.', s=50, lw=0, alpha=0.7,
                     c = colors_list, edgecolor='k')

    for i, label in enumerate(X_labels):
        x,y = X_transform[i]
        if label in teachers_labels:
            axs[0].annotate("TEACHER", (x,y))
    axs[0].set_title('Predicted clustering', fontsize=20)
    
    # Plot showing the actual clusters formed
    cluster_labels_true = [lbl for node, lbl in X_labels_true_clusters]
    colors_list = cm.nipy_spectral(np.asarray(cluster_labels_true).astype(float) / num_classes)
    axs[1].scatter(*zip(*X_transform), marker='.', s=50, lw=0, alpha=0.7,
                     c = colors_list, edgecolor='k')

    for i, label in enumerate(X_labels):
        x,y = X_transform[i]
        if label in teachers_labels:
            axs[1].annotate("TEACHER", (x,y))
    axs[1].set_title('True Clustering', fontsize=15)

    fig.suptitle(in_file.rsplit('/',1)[-1].split('.')[0], fontsize=20)
    plt.show()

In [ ]:
# create Hasse diagram
g_hasse = simplex2hasse_uniform(data, max_order = None)

# perform embedding with the below mentioned parameters
n_walks = 1
w_len = 100
rw_bias = "LOexp"
p_return = 1000000000.
w2v_size = 100
w2v_window = 10
workers = 3

s2v = Simplex2Vec.from_hasse_diagram(g_hasse, n_walks=n_walks, walk_length=w_len, rw_bias = rw_bias,
                                        p = p_return, workers = workers, is_quiet = False)
s2v_model = s2v.fit(size = w2v_size, window = w2v_window)

# cluster the data 
X, X_labels_pred_clusters, X_labels_true_clusters = clusterer(s2v_model, num_classes, 
                                                              teachers_labels = teachers_labels)
# produce NMI of the clustering
NMI = get_NMI(X_labels_pred_clusters, X_labels_true_clusters)

In [ ]:
print("NMI:", NMI)
max_acc_dict = get_max_accuracy_labels(X_labels_pred_clusters, X_labels_true_clusters)
plot_clusterings(X, X_labels_pred_clusters, X_labels_true_clusters, teachers_labels = teachers_labels,
                max_accuracy_dict=max_acc_dict)

# USE CASE

!! Currently under development !! 

In [ ]:
### EXAMPLE: create embeddings using increasingly higher orders without re-building hasse diag.


#Build hasse diagram externally

import numpy as np
import networkx as nx

from Simplex2Vec.Simplex2Vec import Simplex2Vec

from Simplex2Vec.utils import read_simplex_json
from Simplex2Vec.simplex2hasse import *

in_file = "./data/sociopatterns_facets/aggr_5min_cliques_InVS15.json"

data = read_simplex_json(in_file)

In [ ]:
hasse_diag = simplex2hasse_proportional(data, max_order=None)

In [ ]:

for order in range(1,4):
    
    s2v = Simplex2Vec.from_hasse_diagram(hasse_diag, n_walks=1, walk_length=10, hasse_max_order=order)
    
    print('\nMAX ORDER = {}'.format(order))
    print(nx.info(s2v.get_hasse_diagram(copy=False)))
    
    #model = s2v.fit()


In [ ]:
hasse_diag = simplex2hasse_LOadjusted(data, max_order=None)

In [ ]:
for order in range(1,4):
    
    s2v = Simplex2Vec.from_hasse_diagram(hasse_diag, n_walks=1, walk_length=10, hasse_max_order=order)
    
    print('\nMAX ORDER = {}'.format(order))
    print(nx.info(s2v.get_hasse_diagram(copy=False)))
    
    #model = s2v.fit()
    

# Weighted distance matrix of brain data

In [ ]:
import numpy as np
import networkx as nx

from Simplex2Vec.Simplex2Vec import Simplex2Vec
from Simplex2Vec.utils import read_data_HCP
from os.path import join as osjoin

In [ ]:
ICAOrder = 15
data_path = "./data/HCP_Netmats/ICA" + str(ICAOrder) 
#data_path = "/home/jcwbillings/Documents/complexity72h/doublenegroni/data/HCP_PTN1200/netmats_3T_HCP1200_MSMAll_ICAd" + str(ICAOrder) + "_ts2/netmats/3T_HCP1200_MSMAll_d" + str(ICAOrder) + "_ts2"
filename = osjoin(data_path, 'Mnet2.pconn.nii')

data = read_data_HCP(filename)              
# HCP ICA data are delivered as gaussian distributed correlation z-stats. Proceed accordingly
# Take caution for thresholds less than the 50th percentile, self-loops and negative correlations will be included

In [ ]:
s2v = Simplex2Vec.from_graph(data, 
                               threshold=np.percentile(data,55), 
                               n_walks=10, 
                               walk_length=20, 
                               workers=1,
                               hasse_max_order = 3)

In [ ]:
model=s2v.fit()

In [ ]:
model=s2v.refit()

In [ ]:
from ast import literal_eval
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
import matplotlib.cm as cm

X = []
X_labels = []
X_bNodes = []
X_sizes = []
for u in model.wv.vocab.keys():
    X.append(model[u])
    #X_labels.append("".join(list(u)))
    X_labels.append(u)
    X_bNodes.append(literal_eval(X_labels[-1]))
    X_sizes.append(np.size(X_bNodes[-1]))


X_transform = TSNE(n_components=2).fit_transform(X)

In [ ]:
n_clusters = 4

clusterer = KMeans(n_clusters=n_clusters)
cluster_labels = clusterer.fit_predict(X)
clusterer_for_labels = KMeans(n_clusters=n_clusters, random_state=10)
clusterer_for_labels_centers = clusterer_for_labels.fit_predict(X_transform)

colors2 = cm.nipy_spectral(np.arange(n_clusters) / n_clusters)

for hh in range(max(X_sizes)):

    plt.figure(figsize = (12,9))
    ax = plt.gca()
    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax.scatter(*zip(*X_transform), marker='o', s=50, lw=0, alpha=0.7,
                     c = colors, edgecolor='k')
    
    for i, label in enumerate(X_labels):
        x,y = X_transform[i]
        if X_sizes[i]==hh+1:
            ax.annotate(label, (x,y))
    ax.set_title("PCA embedding of the data into 2-dim space")
    plt.show()